# Sugartrail connections

In [113]:
from sugartrail import api, processing, base
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [114]:
%%capture
network_1 = base.Network()
network_2 = base.Network()

1. Insert your [Companies House API](https://developer.company-information.service.gov.uk/how-to-create-an-application) key:

In [115]:
API_input = widgets.Text(
    value='ae444094-0b44-4b5c-b79f-8d8d57ee0116',
    placeholder='Insert API Key',
    disabled=False
)

company_text_1 = widgets.Text(
    value='12420575',
    placeholder='Insert Company ID',
    disabled=True
)

company_text_2 = widgets.Text(
    value='15022841',
    placeholder='Insert Company ID',
    disabled=True
)

auth_status = widgets.HTML(
    value="",
)



auth_button = widgets.Button(description='Authenticate')
auth_button.on_click(lambda bt: auth())

init_button = widgets.Button(description='Initialise', disabled=True)
init_button.on_click(lambda bt: init_network()) 

def auth():
    auth_button.disabled=True
    API_input.disabled=True
    api.basic_auth.username = API_input.value
    if api.test():
        auth_status.value = u'\u2705: Login successful'
        auth_button.button_style='success'
        company_text_1.disabled = False
        company_text_2.disabled = False
        init_button.disabled = False
    else:
        auth_button.disabled=False
        API_input.disabled=False
        auth_status.value = u'\u274c: Invalid API key'

display(API_input, auth_button, auth_status)

Text(value='ae444094-0b44-4b5c-b79f-8d8d57ee0116', placeholder='Insert API Key')

Button(description='Authenticate', style=ButtonStyle())

HTML(value='')

2. Insert the unique company registration numbers (CRNs) for two companies whcih you would like to check are connected:

<!-- Insert a valid [Companies House Public Data API key](https://developer.company-information.service.gov.uk/get-started/) as `username` string value below. If you don't want to use the API and would prefer loading a pre-built network, uncomment and run the cell below and then run the final cell to build and load the map.  -->

In [116]:
init_status = widgets.HTML(
    value="",
) 

depth_selector = widgets.BoundedIntText(
    value=1,
    min=1,
    max=20,
    step=1,
    disabled=True
)

generate_network_button = widgets.Button(description='Build Network', disabled=True)
generate_network_button.on_click(lambda bt: generate_network())

def config_network():
    network_1.hop.officer_appointments_maxsize = 20
    network_1.hop.officers_at_address_maxsize = 20
    network_1.hop.companies_at_address_maxsize = 20

    network_2.hop.officer_appointments_maxsize = 20
    network_1.hop.officers_at_address_maxsize = 20
    network_1.hop.companies_at_address_maxsize = 20
    
    depth_selector.disabled = False
    generate_network_button.disabled = False

def init_network():
    init_button.disabled=True
    response_1 = api.get_company(str(company_text_1.value))
    response_2 = api.get_company(str(company_text_2.value))
    company_text_1.disabled=True
    company_text_2.disabled=True
    if response_1 and response_2:
        network_1.company_id = str(company_text_1.value)
        network_2.company_id = str(company_text_2.value)
        init_status.value = u'\u2705: Initialisation successful for ' + str(response_1['company_name'] + ' and ' + str(response_2['company_name']))
        init_button.button_style='success'
        depth_selector.disabled = False
        generate_network_button.disabled = False
        config_network()
        
    elif not response_1:
        init_button.disabled=False
        company_text_1.disabled=False
        company_text_2.disabled=False
        init_status.value = u'\u274c: Initialisation Failed. No records for company: ' + str(company_text_1.value)

    elif not response_2:
        init_button.disabled=False
        company_text_1.disabled=False
        company_text_2.disabled=False
        init_status.value = u'\u274c: Initialisation Failed. No records for company: ' + str(company_text_2.value)

display(company_text_1, company_text_2, init_button, init_status)

Text(value='12420575', disabled=True, placeholder='Insert Company ID')

Text(value='15022841', disabled=True, placeholder='Insert Company ID')

Button(description='Initialise', disabled=True, style=ButtonStyle())

HTML(value='')

In [117]:
def config_network():
    network_1.hop.officer_appointments_maxsize = 20
    network_1.hop.officers_at_address_maxsize = 20
    network_1.hop.companies_at_address_maxsize = 20

    network_2.hop.officer_appointments_maxsize = 20
    network_1.hop.officers_at_address_maxsize = 20
    network_1.hop.companies_at_address_maxsize = 20
    
    depth_selector.disabled = False
    generate_network_button.disabled = False

3. Select the depth of the networks you would like to build:

In [118]:
build_status = widgets.HTML(
    value="",
)

results = {'connections': []}

show_connections_button = widgets.Button(description='Show Connections', disabled=True)
show_connections_button.on_click(lambda bt: calculate_links())

def generate_network():
    with output_box:
        depth_selector.disabled = True
        results['connections'] = []
        generate_network_button.disabled = True
        results['connections'] = processing.find_network_connections(network_1, network_2, depth_selector.value)
        generate_network_button.button_style = "success"
        if len(results['connections']) > 0:
            for i, connection in enumerate(results['connections']):
                print('Connection ' + str(i + 1) +': ' + connection)
            show_connections_button.disabled = False
        
    
output_box = widgets.Output()
display(depth_selector, generate_network_button, build_status, output_box)

BoundedIntText(value=1, disabled=True, max=20, min=1)

Button(description='Build Network', disabled=True, style=ButtonStyle())

HTML(value='')

Output()

4. Trace the links from each company to the connections:

In [119]:
def calculate_links():
    with links_box:
        for connection in results['connections']:
            print(connection)
            table_data_1 = pd.DataFrame(network_1.find_path(connection))
            table_data_2 = pd.DataFrame(network_2.find_path(connection))
            display(table_data_1)
            display(table_data_2)

    show_connections_button.button_style = "success"

links_box = widgets.Output()
display(show_connections_button, links_box)

Button(description='Show Connections', disabled=True, style=ButtonStyle())

Output()